In [2]:
import numpy as np
import pynbody
import matplotlib.pyplot as plt
import matplotlib as mpl
import pynbody.plot as pp
import pickle
import pandas as pd
import pynbody.plot.sph as sph

In [3]:
from Time import *

In [5]:
#from Load import *

In [35]:
timestep = '002176'
sim_1 ='/home/christenc/Data/Sims/h148.cosmo50PLK.3072g/h148.cosmo50PLK.3072g3HbwK1BH/snapshots/h148.cosmo50PLK.3072g3HbwK1BH.' + timestep
s1 = pynbody.load(sim_1)
h1 = s1.halos()
s1.physical_units()

halo_num = 10
halo = h1.load_copy(halo_num)
halo.physical_units()

In [36]:
pynbody.analysis.halo.center(halo) 

In [42]:
h = h1
rvir_phys = h[halo_num].properties['Rvir']/h[halo_num].properties['h']*h[halo_num].properties['a']
inner_sphere95 = pynbody.filt.Sphere(str(.9*rvir_phys) + ' kpc', [0,0,0])
outer_sphere95 = pynbody.filt.Sphere(str(rvir_phys) + ' kpc', [0,0,0])
shell_part95 = halo[outer_sphere95 & ~inner_sphere95].gas

#Perform calculations
velocity95 = shell_part95['vel'].in_units('kpc yr**-1')
r95 = shell_part95['pos'].in_units('kpc')
Mg95 = shell_part95['mass'].in_units('Msol')
r_mag95 = shell_part95['r'].in_units('kpc')

G_in95 = []     #List of inflowing gas flux per particles at Rvir = .9-1
G_out95 = []     #List of outflowing gas flux per particles at Rvir = .9-1
vr95 = np.sum((velocity95*r95), axis=1)

dL = 0.1*rvir_phys
for x in range(len(vr95)):
    if vr95[x] < 0:
        gin95 = np.array(((vr95[x]/r_mag95[x])*Mg95[x])/dL)
        G_in95 = np.append(G_in95, gin95)
    else:
        gout95 = np.sum(((vr95[x]/r_mag95[x])*Mg95[x])/dL)
        G_out95 = np.append(G_out95, gout95)
#Net flux for inflow and outflow at .95*Rvir
ginflow95 = np.sum(G_in95)
goutflow95 = np.sum(G_out95)


In [38]:
h[halo_num].properties['Rvir']/h[halo_num].properties['h']*h[halo_num].properties['a']

42.26879503859104

In [39]:
max(halo.gas['r'].in_units('kpc'))

42.380969421061046

In [40]:
halo.properties

{'omegaM0': 0.3086,
 'omegaL0': 0.6914,
 'h': 0.6776942783267969,
 'boxsize': Unit("2.95e+04 kpc"),
 'a': 0.5895311905622812,
 'time': Unit("7.50e+00 kpc s km**-1")}

In [43]:
ginflow95

-0.410328965945558

In [44]:
goutflow95

0.07654576543900106